## Подготовка данных

Испортируем все необходимые библиотеки для проекта.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn. preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor, Pool
import lightgbm as lgb
import time 

Рассмотрим строки датасета.

In [2]:
data = pd.read_csv('/datasets/autos.csv')
display(data.head())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Изменим названия столбцов (приведем к нижнему регистру).

In [3]:
data.columns = [x.lower() for x in data.columns]
display(data.head())

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired,datecreated,numberofpictures,postalcode,lastseen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Рассмотрим общую информацию датасета.

In [4]:
print('Размер датасета:', data.shape)
print('')
data.info()

Размер датасета: (354369, 16)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354369 non-null  object
 1   price              354369 non-null  int64 
 2   vehicletype        316879 non-null  object
 3   registrationyear   354369 non-null  int64 
 4   gearbox            334536 non-null  object
 5   power              354369 non-null  int64 
 6   model              334664 non-null  object
 7   kilometer          354369 non-null  int64 
 8   registrationmonth  354369 non-null  int64 
 9   fueltype           321474 non-null  object
 10  brand              354369 non-null  object
 11  repaired           283215 non-null  object
 12  datecreated        354369 non-null  object
 13  numberofpictures   354369 non-null  int64 
 14  postalcode         354369 non-null  int64 
 15  lastseen           354369 non-null  o

Рассмотрим описательную статистику количественных признаков датасета.

In [5]:
data.describe()

,price,registrationyear,power,kilometer,registrationmonth,numberofpictures,postalcode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Как можно заметить, выделяются аномалии в признаках "registrationyear", "power" и "price". Год регистрации авто не может быть больше 2023  и не может быть равен 1000. Примем минимальный год регистрации автомобиля 1980. Значение мощности автомобиля max равно 20000, и не может превышать 3000 л.с.(максимальная известная мощность двигателя в л.с.). Также аномально низко выглядит цена за автомобиль со значением 0. Примем, что минимальная стоимость автомобился равна 500 евро.

Считаю необходимым убрать признаки "numberofpictures", так как данный признак практически во всех строках имеет значение 0, признак "postalcode", так как почтовый индекс владельца анкеты не влияет на стоимость автомобиля. Также считаю, что признаки "datecrawled", "datecreated" и "lastseen" не влияют на стоимость автомобиля в виду нулевого влияния на его состояние. Также считаю, что признак "registrationmonth" не настолько важен, как "registrationyear" и можно им принебречь.     

С учетом этого, обновим наш датасет.

In [6]:
data = data.query("1980 < registrationyear < 2023 & power < 3000 & price > 500")
data = data.drop(
    [
        "datecrawled",
        "datecreated",
        "lastseen",
        "registrationyear",
        "numberofpictures",
        "postalcode",
    ],
    axis=1,
)
data = data.reset_index(drop=True)
display(data.head())
print("Размер датасета:", data.shape)

,price,vehicletype,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
0,18300,coupe,manual,190,NaN,125000,5,gasoline,audi,yes
1,9800,suv,auto,163,grand,125000,8,gasoline,jeep,NaN
2,1500,small,manual,75,golf,150000,6,petrol,volkswagen,no
3,3600,small,manual,69,fabia,90000,7,gasoline,skoda,no
4,650,sedan,manual,102,3er,150000,10,petrol,bmw,yes


Размер датасета: (309177, 10)


Посчитаем количество пропусков в датасете.

In [7]:
data.isna().sum()

price                    0
vehicletype          25814
gearbox              12278
power                    0
model                13446
kilometer                0
registrationmonth        0
fueltype             21883
brand                    0
repaired             51993
dtype: int64

Заполним пропуски в признаке "repaired" заглушкой unknown  

In [8]:
data['repaired'] = data['repaired']. fillna('unknown')
data['model'] = data['model']. fillna('unknown')

In [9]:
print('Размер датасета:', data.shape)
print('')
data.isna().sum()

Размер датасета: (309177, 10)



price                    0
vehicletype          25814
gearbox              12278
power                    0
model                    0
kilometer                0
registrationmonth        0
fueltype             21883
brand                    0
repaired                 0
dtype: int64

Посчитаем количество нулевых значений в признаке "power".

In [10]:
len(data[data['power'] == 0])

26637

Рассчитаем медиану признака "power".

In [11]:
data['power'].median()

109.0

Заменим нули в признаке "power" на медианное значение.

In [12]:
data = data.replace([0],[data['power'].median()])

Посчитаем количество нулевых значений признака "power".

In [13]:
len(data[data['power'] == 0])

0

In [14]:
print('Размер датасета:', data.shape)

Размер датасета: (309177, 10)


In [15]:
data["vehicletype"] = data.groupby("model")["vehicletype"].transform(
    lambda x: x.fillna((x.mode()[0] if x.count() != 0 else "unknown"))
)
data["gearbox"] = data.groupby("model")["gearbox"].transform(
    lambda x: x.fillna((x.mode()[0] if x.count() != 0 else "unknown"))
)
data["fueltype"] = data.groupby("model")["fueltype"].transform(
    lambda x: x.fillna((x.mode()[0] if x.count() != 0 else "unknown"))
)

In [16]:
data.isna().sum()

price                0
vehicletype          0
gearbox              0
power                0
model                0
kilometer            0
registrationmonth    0
fueltype             0
brand                0
repaired             0
dtype: int64

In [17]:
print(data.isna().sum())

price                0
vehicletype          0
gearbox              0
power                0
model                0
kilometer            0
registrationmonth    0
fueltype             0
brand                0
repaired             0
dtype: int64


Так как сумма оставшихся пропусков меньше 10% от общего количества строк в датасете, удалим данные пропуски.

In [18]:
print(data.isna().sum())
display(data.head())
print('')
print('Размер датасета:', data.shape)

price                0
vehicletype          0
gearbox              0
power                0
model                0
kilometer            0
registrationmonth    0
fueltype             0
brand                0
repaired             0
dtype: int64


,price,vehicletype,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
0,18300.0,coupe,manual,190.0,unknown,125000.0,5.0,gasoline,audi,yes
1,9800.0,suv,auto,163.0,grand,125000.0,8.0,gasoline,jeep,unknown
2,1500.0,small,manual,75.0,golf,150000.0,6.0,petrol,volkswagen,no
3,3600.0,small,manual,69.0,fabia,90000.0,7.0,gasoline,skoda,no
4,650.0,sedan,manual,102.0,3er,150000.0,10.0,petrol,bmw,yes



Размер датасета: (309177, 10)


<b>Вывод: </b>
<br> На данной этапе было выполнена обработка датасета. Были удалены признаки, не влияющие на формирование целевого признака. На осовании описательной статистики были определены выбросы и аномальные значения в количественных признаках. Также удалены строки с пропусками в категориальных признаках, так как процентное соотношение этих пропусков к общему размеру датасета не превышает 10%.

## Обучение моделей

Проведем стандартизацию количественных признаков, сперва создав три выборки: тренировочную, валидационную и тестовую.

In [19]:
pd.options.mode.chained_assignment = None

for col in data.select_dtypes(include=["object"]):
    data[col] = data[col].astype("category")

target = data["price"]
features = data.drop("price", axis=1)
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345
)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345
)

numeric = ["power", "kilometer", "registrationmonth"]

scaler = StandardScaler()
features_train[numeric] = scaler.fit_transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

print('Размер "features_train":', features_train.shape)
print('Размер "features_valid":', features_valid.shape)
print('Размер "features_test":', features_test.shape)
features_train = features_train.reset_index().drop(["index"], axis=1)
features_valid = features_valid.reset_index().drop(["index"], axis=1)
features_test = features_test.reset_index().drop(["index"], axis=1)
display(features_train.head())

Размер "features_train": (185506, 9)
Размер "features_valid": (61835, 9)
Размер "features_test": (61836, 9)


,vehicletype,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
0,bus,auto,0.528575,c4,-2.670185,-0.150276,petrol,citroen,unknown
1,sedan,manual,0.864240,vectra,0.589188,-0.150276,gasoline,opel,unknown
2,bus,auto,0.432671,vito,0.589188,-0.334437,gasoline,mercedes_benz,yes
3,wagon,manual,0.033070,passat,0.589188,-0.150276,petrol,volkswagen,no
4,wagon,manual,-0.350547,passat,0.589188,-0.150276,petrol,volkswagen,no


Закодируем категориальные признаки как однократный числовой массив.

In [20]:
# создадим OneHotEncoder

encoder = OneHotEncoder(handle_unknown="ignore")
features_train_ohe = pd.DataFrame(
    encoder.fit_transform(
        features_train[
            ["vehicletype", "gearbox", "model", "fueltype", "brand", "repaired"]
        ]
    ).toarray()
)
features_valid_ohe = pd.DataFrame(
    encoder.transform(
        features_valid[
            ["vehicletype", "gearbox", "model", "fueltype", "brand", "repaired"]
        ]
    ).toarray()
)
features_test_ohe = pd.DataFrame(
    encoder.transform(
        features_test[
            ["vehicletype", "gearbox", "model", "fueltype", "brand", "repaired"]
        ]
    ).toarray()
)
features_train_ohe = features_train.join(features_train_ohe)
features_train_ohe = features_train_ohe.drop(
    ["vehicletype", "gearbox", "model", "fueltype", "brand", "repaired"], axis=1
)
features_valid_ohe = features_valid.join(features_valid_ohe)
features_valid_ohe = features_valid_ohe.drop(
    ["vehicletype", "gearbox", "model", "fueltype", "brand", "repaired"], axis=1
)
features_test_ohe = features_test.join(features_test_ohe)
features_test_ohe = features_test_ohe.drop(
    ["vehicletype", "gearbox", "model", "fueltype", "brand", "repaired"], axis=1
)
display(features_train_ohe.head())
display(features_valid_ohe.head())
display(features_train_ohe.head())

,power,kilometer,registrationmonth,0,1,2,3,4,5,6,...,299,300,301,302,303,304,305,306,307,308
0,0.528575,-2.670185,-0.150276,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.864240,0.589188,-0.150276,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.432671,0.589188,-0.334437,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.033070,0.589188,-0.150276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,-0.350547,0.589188,-0.150276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


,power,kilometer,registrationmonth,0,1,2,3,4,5,6,...,299,300,301,302,303,304,305,306,307,308
0,-1.245653,0.589188,-0.113444,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.222674,-1.312113,-0.444933,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.846052,0.589188,-0.408101,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-1.005892,-2.398570,-0.444933,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.640464,0.589188,-0.076612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


,power,kilometer,registrationmonth,0,1,2,3,4,5,6,...,299,300,301,302,303,304,305,306,307,308
0,0.528575,-2.670185,-0.150276,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.864240,0.589188,-0.150276,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.432671,0.589188,-0.334437,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.033070,0.589188,-0.150276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,-0.350547,0.589188,-0.150276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


С помощью GridSearchCV определим лучшие параметры по параметру MSE для модели DecisionTreeRegressor.

In [21]:
dtr = DecisionTreeRegressor(random_state=12345)
parametrs = {
    "max_depth": range(1, 6),
    "min_samples_split": range(2, 10, 2),
    "min_samples_leaf": range(2, 10, 2),
}
grid = GridSearchCV(
    dtr, parametrs, cv=5, n_jobs=-1, verbose=10, scoring="neg_root_mean_squared_error"
)
grid.fit(features_train_ohe, target_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV 1/5; 1/80] START max_depth=1, min_samples_leaf=2, min_samples_split=2.......
[CV 1/5; 1/80] END max_depth=1, min_samples_leaf=2, min_samples_split=2; total time=   0.7s
[CV 2/5; 1/80] START max_depth=1, min_samples_leaf=2, min_samples_split=2.......
[CV 2/5; 1/80] END max_depth=1, min_samples_leaf=2, min_samples_split=2; total time=   0.8s
[CV 3/5; 1/80] START max_depth=1, min_samples_leaf=2, min_samples_split=2.......
[CV 3/5; 1/80] END max_depth=1, min_samples_leaf=2, min_samples_split=2; total time=   0.7s
[CV 4/5; 1/80] START max_depth=1, min_samples_leaf=2, min_samples_split=2.......
[CV 4/5; 1/80] END max_depth=1, min_samples_leaf=2, min_samples_split=2; total time=   0.7s
[CV 5/5; 1/80] START max_depth=1, min_samples_leaf=2, min_samples_split=2.......
[CV 5/5; 1/80] END max_depth=1, min_samples_leaf=2, min_samples_split=2; total time=   0.7s
[CV 1/5; 2/80] START max_depth=1, min_samples_leaf=2, min_samples_split=4

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=12345),
             n_jobs=-1,
             param_grid={'max_depth': range(1, 6),
                         'min_samples_leaf': range(2, 10, 2),
                         'min_samples_split': range(2, 10, 2)},
             scoring='neg_root_mean_squared_error', verbose=10)

Рассмотрим лучшие параметры.

In [22]:
grid.best_params_ 

{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}

Определим RMSE для тренировочной выборки при использовании модели DecisionTreeRegressor.

In [23]:
rmse = - grid.best_score_
print('RMSE равно:"', rmse)

RMSE равно:" 2982.9662141186905


Определим время обучения при использовании модели DecisionTreeRegressor.

In [24]:
start = time.time()

model = DecisionTreeRegressor(max_depth = 5, min_samples_split = 2, min_samples_leaf = 2, random_state=12345)
model.fit(features_train_ohe, target_train)

end_fit_dtr = time.time() - start

Определим время предсказания при использовании модели DecisionTreeRegressor и качество модели.

In [25]:
start = time.time()

predictions = model.predict(features_train_ohe)
rmse_dtr = mean_squared_error(target_train, predictions) ** 0.5

end_predict_dtr = time.time() - start

print(rmse_dtr)

2983.7739720728337


Зададим список качественных признаков.

In [26]:
cat_features = ["vehicletype", "gearbox", "model", "fueltype", "brand", "repaired"]

In [27]:
cb = CatBoostRegressor(n_estimators=1500,
                       loss_function='RMSE',
                       learning_rate=0.4,
                       depth=3, task_type='CPU',
                       random_state=1,
                       verbose=False)

pool_train = Pool(features_train, target_train,
                  cat_features = cat_features)

pool_valid = Pool(features_valid, cat_features = cat_features)

Определим время обучения модели CatBoostRegressor.

In [28]:
start = time.time()

cb.fit(pool_train, verbose=30) 

end_fit_cb = time.time() - start

0:	learn: 3851.3311712	total: 199ms	remaining: 4m 58s
30:	learn: 2628.4752928	total: 2.83s	remaining: 2m 14s
60:	learn: 2533.8226392	total: 5.41s	remaining: 2m 7s
90:	learn: 2490.4805384	total: 8.14s	remaining: 2m 6s
120:	learn: 2454.1915877	total: 10.8s	remaining: 2m 3s
150:	learn: 2428.1013549	total: 13.4s	remaining: 1m 59s
180:	learn: 2407.6209693	total: 16.3s	remaining: 1m 58s
210:	learn: 2392.6794172	total: 19s	remaining: 1m 56s
240:	learn: 2377.1873525	total: 21.4s	remaining: 1m 51s
270:	learn: 2364.5795354	total: 23.8s	remaining: 1m 47s
300:	learn: 2349.9015982	total: 26.6s	remaining: 1m 46s
330:	learn: 2337.6074834	total: 29.3s	remaining: 1m 43s
360:	learn: 2332.1908061	total: 31.7s	remaining: 1m 40s
390:	learn: 2323.5258518	total: 34.5s	remaining: 1m 37s
420:	learn: 2314.6192535	total: 37.2s	remaining: 1m 35s
450:	learn: 2309.5688143	total: 39.7s	remaining: 1m 32s
480:	learn: 2302.1469412	total: 42.3s	remaining: 1m 29s
510:	learn: 2296.9765753	total: 44.7s	remaining: 1m 26s
54

Определим время предсказания при использовании модели CatBoostRegressor и качество модели.

In [29]:
start = time.time()

predictions = cb.predict(features_valid)
rmse_cb = mean_squared_error(target_valid, predictions) ** 0.5

end_predict_cb = time.time() - start

print(rmse_cb)

2239.7797697593455


Параметра модели LightGBM.

In [30]:
params = {
    'num_iterations': 500,
    'max_depth': 20,
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learnnig_rage': 0.05,
    'metric': 'l1',
    'verbose': -1
}

Создадим данные для модели LightGBM.

In [31]:
lgb_train = lgb.Dataset(features_train, target_train)
lgb_valid= lgb.Dataset(features_valid, target_valid)

Определим время обучения модели LightGBM.

In [32]:
start = time.time()

start = time.time()
model = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_valid,
                 early_stopping_rounds=10)
end_train_lgb = time.time() - start

end_fit_lgbm = time.time() - start

[LightGBM] [Warning] Unknown parameter: learnnig_rage


/opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's l1: 3375.2
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l1: 3216.28
[3]	valid_0's l1: 3075.91
[4]	valid_0's l1: 2955.68
[5]	valid_0's l1: 2848.14
[6]	valid_0's l1: 2756.86
[7]	valid_0's l1: 2672.29
[8]	valid_0's l1: 2594.87
[9]	valid_0's l1: 2523.89
[10]	valid_0's l1: 2464.44
[11]	valid_0's l1: 2410.51
[12]	valid_0's l1: 2363.58
[13]	valid_0's l1: 2314.38
[14]	valid_0's l1: 2268.59
[15]	valid_0's l1: 2227.24
[16]	valid_0's l1: 2191.72
[17]	valid_0's l1: 2156.16
[18]	valid_0's l1: 2124.45
[19]	valid_0's l1: 2093.73
[20]	valid_0's l1: 2070.17
[21]	valid_0's l1: 2045.74
[22]	valid_0's l1: 2026.19
[23]	valid_0's l1: 2005.2
[24]	valid_0's l1: 1987.55
[25]	valid_0's l1: 1967.59
[26]	valid_0's l1: 1952.85
[27]	valid_0's l1: 1938.39
[28]	valid_0's l1: 1924.83
[29]	valid_0's l1: 1908.94
[30]	valid_0's l1: 1897.86
[31]	valid_0's l1: 1887.56
[32]	valid_0's l1: 1876.69
[33]	valid_0's l1: 1868.7
[34]	valid_0's l1: 1859.73
[35]	valid_0's l1: 1851.47
[36]	v

Определим время предсказания модели и качество.

In [33]:
start = time.time()

predictions = model.predict(features_valid)
rmse_lgbm = mean_squared_error(target_valid, predictions) ** 0.5

end_predict_lgbm = time.time() - start

print(rmse_lgbm)

2215.9240206661857


## Анализ моделей

Для анализа модели создадим датафрейм с данными: название модели, время обучения, время предсказания, значение метрики качества.

In [34]:
df = pd.DataFrame({'Название модели': ['DecisionTreeRegressor.', 'CatBoostRegressor', 'LightGBM'], 
                   'Время обучения':     [end_fit_dtr, end_fit_cb, end_fit_lgbm], 
                   'Время предсказания': [end_predict_dtr, end_predict_cb, end_predict_lgbm],
                   'Общее время':        [end_fit_dtr + end_predict_dtr, 
                                          end_fit_cb + end_predict_cb, 
                                          end_fit_lgbm + end_predict_lgbm],
                   'Значение метрики RMSE': [rmse_dtr, rmse_cb, rmse_lgbm]})
display(df)

,Название модели,Время обучения,Время предсказания,Общее время,Значение метрики RMSE
0,DecisionTreeRegressor.,2.255437,0.220686,2.476124,2983.773972
1,CatBoostRegressor,128.779253,0.407532,129.186785,2239.779770
2,LightGBM,169.050292,1.915229,170.965521,2215.924021


Как можно заметить наилучшее значение метрики RMSE у модели LightGBM, также можно заметить, что данная модель имеет второе самое быстрое общее время обучения и предсказания. Протестируем модель LightGBM на тестовой выборке.

In [35]:
model = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_valid,
                 early_stopping_rounds=10)

[LightGBM] [Warning] Unknown parameter: learnnig_rage


/opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l1: 3375.2
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l1: 3216.28
[3]	valid_0's l1: 3075.91
[4]	valid_0's l1: 2955.68
[5]	valid_0's l1: 2848.14
[6]	valid_0's l1: 2756.86
[7]	valid_0's l1: 2672.29
[8]	valid_0's l1: 2594.87
[9]	valid_0's l1: 2523.89
[10]	valid_0's l1: 2464.44
[11]	valid_0's l1: 2410.51
[12]	valid_0's l1: 2363.58
[13]	valid_0's l1: 2314.38
[14]	valid_0's l1: 2268.59
[15]	valid_0's l1: 2227.24
[16]	valid_0's l1: 2191.72
[17]	valid_0's l1: 2156.16
[18]	valid_0's l1: 2124.45
[19]	valid_0's l1: 2093.73
[20]	valid_0's l1: 2070.17
[21]	valid_0's l1: 2045.74
[22]	valid_0's l1: 2026.19
[23]	valid_0's l1: 2005.2
[24]	valid_0's l1: 1987.55
[25]	valid_0's l1: 1967.59
[26]	valid_0's l1: 1952.85
[27]	valid_0's l1: 1938.39
[28]	valid_0's l1: 1924.83
[29]	valid_0's l1: 1908.94
[30]	valid_0's l1: 1897.86
[31]	valid_0's l1: 1887.56
[32]	valid_0's l1: 1876.69
[33]	valid_0's l1: 1868.7
[34]	valid_0's l1: 1859.73
[35]	valid_0's l1: 1851.47
[36]	v

In [36]:
predictions = model.predict(features_test)
rmse_lgbm = mean_squared_error(target_test, predictions) ** 0.5
print(rmse_lgbm)

2224.190661805113


Как можно заметить, значение метрики чуть выше требуемой.

<b>Вывод: </b>

В данной работе были проведено исследование моделей на быстроту обучения и точность предсказания. Были выбраны три модели DecisionTreeRegressor, CatBoostRegressor, LightGBM. Предварительно данные для исследования были обработаны от выбросов и аномальных значений, а также были удалены признаки, которые не влияют на целевой признак. В результате исследования была выбрана модель LightGBM, так как она удовлетворяла по значению метрики, как не тренировочной так и на тестовой, но имеет большое время обучения.